### Importing libraries

In [2]:
#general
import pandas as pd

#calculating geo distance
import geopandas as gpd
from shapely.geometry import Point

#importing data from URL
import requests

#allowing users to decide which functionality they prefer
import argparse

#allowing users to input interest points with flexibility 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#nice displaying of results in terminal
from tabulate import tabulate

### Importing function for calculating geo distance

In [3]:
# transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
def to_mercator(lat, long):  
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

# return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
def distance_meters(start, finish):
    return start.distance(finish)

### Extracting raw data - Conecting to BiciMad API

In [4]:
#It is possible to take frozen data from january 2022 using:
#bicimad_stations=pd.read_csv('../data/bicimad_stations.csv')
#but in order to have fresh data API connection is included

In [5]:
# getting token
login_url = 'https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/'
login_headers = {'email': 'malva.gonzalez@ironhack.com','password': 'Bicimad2022'}
accessToken=requests.get(login_url,headers=login_headers).json()['data'][0]['accessToken']

In [6]:
#Getting stations information
stations_url = 'https://openapi.emtmadrid.es/v1/transport/bicimad/stations/'
stations_headers = {'accessToken':accessToken }
bicimad_stations_json=requests.get(stations_url,headers=stations_headers).json()['data']

In [7]:
bicimad_stations=pd.json_normalize(bicimad_stations_json)

Taking only the intersting columns needed for the project and filtering the data:

In [8]:
bicimad_stations=bicimad_stations[["address","geometry.coordinates","name","dock_bikes","free_bases"]]

In [9]:
bicimad_stations = bicimad_stations.rename(columns={'geometry.coordinates': 'geometry_coordinates'})

Filtering out rows without geometry_coordinates (distance can not be calculated, in this case there are not null values):

In [11]:
bicimad_stations=bicimad_stations[bicimad_stations.geometry_coordinates.notnull()]

Checking if stations name are unique (they are not but the address is different so this is not a problem):

In [12]:
bicimad_stations.groupby("name").filter(lambda x: len(x) > 1).sort_values("name")

,address,geometry_coordinates,name,dock_bikes,free_bases
129,Calle Fernando el Católico nº 19,"[-3.708439, 40.4338516]",Fernando el Católico,16,7
201,Calle Fernando el Católico nº 61,"[-3.7149166, 40.4343611]",Fernando el Católico,5,20
89,Calle Goya nº 18,"[-3.6837876, 40.4250361]",General Pardiñas,8,18
188,Calle General Pardiñas nº 56,"[-3.6782777, 40.4290555]",General Pardiñas,12,11
121,Calle Méndez Álvaro nº 11,"[-3.6863218, 40.4013216]",Méndez Álvaro,3,20
182,Calle Retama nº 5,"[-3.6758888, 40.3941388]",Méndez Álvaro,4,19
165,Paseo de la Esperanza nº 2,"[-3.7064516, 40.4035988]",Paseo de la Esperanza,6,8
172,Paseo de la Esperanza nº 21,"[-3.7043611, 40.401]",Paseo de la Esperanza,10,12


Calculating coordinates:

In [13]:
def left_split(x):
    return x[0]
def right_split(x):
    return x[-1]

In [14]:
long_finish=bicimad_stations.apply(lambda x: left_split(x['geometry_coordinates']), axis=1)

In [15]:
lat_finish=bicimad_stations.apply(lambda x: right_split(x['geometry_coordinates']), axis=1)

In [16]:
bicimad_stations["long_finish"]=long_finish

In [17]:
bicimad_stations["lat_finish"]=lat_finish

In [18]:
bicimad_stations

,address,geometry_coordinates,name,dock_bikes,free_bases,long_finish,lat_finish
0,Puerta del Sol nº 1,"[-3.7018341, 40.4172137]",Puerta del Sol A,18,8,-3.701834,40.417214
1,Puerta del Sol nº 1,"[-3.701602938060457, 40.41731271011562]",Puerta del Sol B,21,7,-3.701603,40.417313
2,Calle Miguel Moya nº 1,"[-3.7058415, 40.4205886]",Miguel Moya,11,12,-3.705842,40.420589
3,Plaza del Conde del Valle de Súchil nº 3,"[-3.7069171, 40.4302937]",Plaza Conde Suchil,14,2,-3.706917,40.430294
4,Calle Manuela Malasaña nº 5,"[-3.7025875, 40.4285524]",Malasaña,16,6,-3.702587,40.428552
...,...,...,...,...,...,...,...
259,Avenida Juan de Herrera frente a la calle Paul...,"[-3.72997, 40.43896]",INEF,6,17,-3.729970,40.438960
260,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72699, 40.44375]",Ciudad Universitaria 1,0,24,-3.726990,40.443750
261,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72693, 40.44342]",Ciudad Universitaria 2,9,15,-3.726930,40.443420
262,Calle José Antonio Novais frente al nº 12,"[-3.7272945, 40.4483322]",Facultad Biología,17,6,-3.727295,40.448332


Renaming field names:

In [19]:
bicimad_stations = bicimad_stations.rename(columns={'address': 'Station location', 'name': 'BiciMAD station'})

Deleting not needed columns:

In [20]:
bicimad_stations = bicimad_stations.drop(columns='geometry_coordinates')

Calculating coordenades:

In [21]:
bicimad_stations[["long_finish", "lat_finish"]] = bicimad_stations[["long_finish", 
                                                                   "lat_finish"]].apply(pd.to_numeric)

In [22]:
%%time
coordinates_bicimad=bicimad_stations.apply(lambda x: to_mercator(x['lat_finish'],
                                                              x['long_finish']), axis=1)

CPU times: user 6.81 s, sys: 516 ms, total: 7.33 s
Wall time: 7.4 s


In [23]:
bicimad_stations["coordinates_bicimad"]=coordinates_bicimad

In [24]:
bicimad_stations.drop(['long_finish','lat_finish'], axis=1, inplace=True)

In [25]:
bicimad_stations

,Station location,BiciMAD station,dock_bikes,free_bases,coordinates_bicimad
0,Puerta del Sol nº 1,Puerta del Sol A,18,8,POINT (4499223.648366921 -412373.2851572837)
1,Puerta del Sol nº 1,Puerta del Sol B,21,7,POINT (4499234.670122574 -412347.498528782)
2,Calle Miguel Moya nº 1,Miguel Moya,11,12,POINT (4499599.340516399 -412820.3206106317)
3,Plaza del Conde del Valle de Súchil nº 3,Plaza Conde Suchil,14,2,POINT (4500679.707306497 -412940.3068148262)
4,Calle Manuela Malasaña nº 5,Malasaña,16,6,POINT (4500485.866677178 -412457.3286497012)
...,...,...,...,...,...
259,Avenida Juan de Herrera frente a la calle Paul...,INEF,6,17,POINT (4501644.435409558 -415511.9577777153)
260,Avenida de la Complutense (Metro Ciudad Univer...,Ciudad Universitaria 1,0,24,POINT (4502177.655770458 -415179.5220676512)
261,Avenida de la Complutense (Metro Ciudad Univer...,Ciudad Universitaria 2,9,15,POINT (4502140.920338497 -415172.8287427756)
262,Calle José Antonio Novais frente al nº 12,Facultad Biología,17,6,POINT (4502687.743941171 -415213.4906984274)


### Extracting raw data -  Interesting Points

Defining public url for accesing to the datasets:

In [81]:
url_interest_points = 'https://datos.madrid.es/egob/catalogo/300356-0-monumentos-ciudad-madrid.json'

Taking url information with requests:

In [82]:
response_interest_points = requests.get(url_interest_points)

In [83]:
response_interest_points=response_interest_points.json()['@graph']

Converting json to pandas dataframe and taking only needed fields:

In [84]:
interest_points = pd.json_normalize(response_interest_points)

In [85]:
interest_points

,@id,id,title,relation,references,address.district.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.organization-name,address.area.@id
0,https://datos.madrid.es/egob/catalogo/tipo/mon...,409634,A las víctimas del Holocausto,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,JARDIN TRES CULTURAS,40.464322,-3.603558,"La obra, creada por el escultor judío de orige...",A las víctimas del Holocausto,NaN
1,https://datos.madrid.es/egob/catalogo/tipo/mon...,409512,A los Abuelos,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Alicún,40.479812,-3.661719,"Pequeña escultura, situada en un parque delimi...",A los Abuelos,https://datos.madrid.es/egob/kos/Provincia/Mad...
2,https://datos.madrid.es/egob/catalogo/tipo/mon...,408781,A Neruda-El Ser alado,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,Gta José Luis Ozores 116,40.381855,-3.652419,Monumento dedicado al escritor Pablo Neruda (1...,A Neruda-El Ser alado,https://datos.madrid.es/egob/kos/Provincia/Mad...
3,https://datos.madrid.es/egob/catalogo/tipo/mon...,408836,Abogados Laboristas,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Atocha 55,40.412911,-3.700017,El 24 de enero de 1977 se produjo un atentado ...,Abogados Laboristas,https://datos.madrid.es/egob/kos/Provincia/Mad...
4,https://datos.madrid.es/egob/catalogo/tipo/mon...,408920,Abstracta I,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,,40.421746,-3.662293,"La histórica quinta de la Fuente del Berro, de...",Abstracta I,https://datos.madrid.es/egob/kos/Provincia/Mad...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1843,https://datos.madrid.es/egob/catalogo/tipo/mon...,411003,XXV aniversario Colegio Mayor Argentino,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Martín Fierro,40.436127,-3.731997,"La lápida, promovida por la Embajada de la Rep...",XXV aniversario Colegio Mayor Argentino,https://datos.madrid.es/egob/kos/Provincia/Mad...
1844,https://datos.madrid.es/egob/catalogo/tipo/mon...,411004,XXV aniversario Colegio Mayor Argentino,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Martín Fierro,40.436100,-3.732009,"La lápida, promovida por el Ministerio de Cult...",XXV aniversario Colegio Mayor Argentino,https://datos.madrid.es/egob/kos/Provincia/Mad...
1845,https://datos.madrid.es/egob/catalogo/tipo/mon...,408801,Yawar Fiesta,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,NaN,NaN,,,NaN,NaN,El Colegio Público Eduardo Rosales pertenece a...,Yawar Fiesta,NaN
1846,https://datos.madrid.es/egob/catalogo/tipo/mon...,409054,'Sin título',https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,,NaN,NaN,"La obra, del escultor madrileño José Miguel Ut...",'Sin título',https://datos.madrid.es/egob/kos/Provincia/Mad...


In [86]:
interest_points=interest_points[["title","address.street-address","location.latitude","location.longitude","organization.organization-desc"]]

In [87]:
interest_points

,title,address.street-address,location.latitude,location.longitude,organization.organization-desc
0,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,"La obra, creada por el escultor judío de orige..."
1,A los Abuelos,C Alicún,40.479812,-3.661719,"Pequeña escultura, situada en un parque delimi..."
2,A Neruda-El Ser alado,Gta José Luis Ozores 116,40.381855,-3.652419,Monumento dedicado al escritor Pablo Neruda (1...
3,Abogados Laboristas,C Atocha 55,40.412911,-3.700017,El 24 de enero de 1977 se produjo un atentado ...
4,Abstracta I,,40.421746,-3.662293,"La histórica quinta de la Fuente del Berro, de..."
...,...,...,...,...,...
1843,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,40.436127,-3.731997,"La lápida, promovida por la Embajada de la Rep..."
1844,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,40.436100,-3.732009,"La lápida, promovida por el Ministerio de Cult..."
1845,Yawar Fiesta,,NaN,NaN,El Colegio Público Eduardo Rosales pertenece a...
1846,'Sin título',,NaN,NaN,"La obra, del escultor madrileño José Miguel Ut..."


Renaming and adding columns needed:

In [88]:
interest_points = interest_points.rename(columns=
{'title': 'Place of interest', 'address.street-address': 'Place address',
 'location.latitude': 'lat_start','location.longitude': 'long_start','organization.organization-desc': 'Place Description'})

In [89]:
interest_points["Type of place"] = "Monumentos de la ciudad de Madrid" 

Cleaning the dataframe (excluding rows without latitude or longitude information):

In [90]:
interest_points=interest_points[interest_points.lat_start.notnull()]

In [91]:
interest_points=interest_points[interest_points.long_start.notnull()]

Cheking if points of interest are unique (names are not unique so minim level of detail is name+address):

In [92]:
interest_points.groupby("Place of interest").filter(lambda x: len(x) > 1).sort_values("Place of interest")

,Place of interest,Place address,lat_start,long_start,Place Description,Type of place
12,"Agricultor, agricultura y progreso",Avda Complutense,40.441972,-3.729384,"Grupo escultórico de moderna estética, denomin...",Monumentos de la ciudad de Madrid
13,"Agricultor, agricultura y progreso",C Principal de Provincias,40.412532,-3.739214,"Este grupo escultórico, de moderna y conceptua...",Monumentos de la ciudad de Madrid
14,"Agricultor, agricultura y progreso",PASEO CASTELLANA 101,40.453458,-3.691112,Con ocasión de un congreso realizado en este p...,Monumentos de la ciudad de Madrid
37,Alfonso X El Sabio,PASEO RECOLETOS 20,40.423004,-3.690836,Al aproximarse la finalización de las obras d...,Monumentos de la ciudad de Madrid
38,Alfonso X El Sabio,JARDINES SABATINI,40.421540,-3.713312,Forma parte esta escultura de la serie concebi...,Monumentos de la ciudad de Madrid
...,...,...,...,...,...,...
1823,Vicente Aleixandre,Gta Presidente García Moreno,40.445849,-3.714497,Inaugurada el 10 de diciembre de 1990 siendo a...,Monumentos de la ciudad de Madrid
1843,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,40.436127,-3.731997,"La lápida, promovida por la Embajada de la Rep...",Monumentos de la ciudad de Madrid
1844,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,40.436100,-3.732009,"La lápida, promovida por el Ministerio de Cult...",Monumentos de la ciudad de Madrid
50,Álvaro Iglesias Sánchez,CALLE CARRANZA 7,40.429000,-3.703131,El 6 de abril de 1982 se produjo un incendio e...,Monumentos de la ciudad de Madrid


Cheking if there are duplicates by name and address (6 duplicates, there will be cleaned later):

In [93]:
interest_duplicates=interest_points.groupby(["Place of interest","Place address"]).filter(lambda x: len(x) > 1).sort_values("Place of interest")[["Place of interest","Place address"]]

In [94]:
(interest_duplicates["Place of interest"]+interest_duplicates["Place address"]).unique()

array(['Antonio BienvenidaC Alcalá 231',
       'Casa de la PanaderíaPza Mayor 27', 'Enrique Tierno Galván ',
       'Miguel de CervantesC Cervantes 2',
       'Rey astur-leonésC Méndez Núñez 1',
       'XXV aniversario Colegio Mayor ArgentinoC Martín Fierro '],
      dtype=object)

Calculating coordinades:

In [95]:
interest_points[["long_start", "lat_start"]] = interest_points[["long_start", 
                                                                   "lat_start"]].apply(pd.to_numeric)

In [96]:
%%time
coordinates_interest_points=interest_points.apply(lambda x: to_mercator(x['lat_start'],
                                                              x['long_start']), axis=1)

CPU times: user 44.4 s, sys: 2.91 s, total: 47.3 s
Wall time: 47.4 s


In [97]:
interest_points["coordinates_interest_points"]=coordinates_interest_points

In [98]:
interest_points.drop(['long_start','lat_start'], axis=1, inplace=True)

In [99]:
interest_points

,Place of interest,Place address,Place Description,Type of place,coordinates_interest_points
0,A las víctimas del Holocausto,JARDIN TRES CULTURAS,"La obra, creada por el escultor judío de orige...",Monumentos de la ciudad de Madrid,POINT (4504467.690585745 -401410.9402809891)
1,A los Abuelos,C Alicún,"Pequeña escultura, situada en un parque delimi...",Monumentos de la ciudad de Madrid,POINT (4506192.045251724 -407898.4486445754)
2,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumento dedicado al escritor Pablo Neruda (1...,Monumentos de la ciudad de Madrid,POINT (4495287.585088818 -406861.0266344676)
3,Abogados Laboristas,C Atocha 55,El 24 de enero de 1977 se produjo un atentado ...,Monumentos de la ciudad de Madrid,POINT (4498744.636240896 -412170.5648424985)
4,Abstracta I,,"La histórica quinta de la Fuente del Berro, de...",Monumentos de la ciudad de Madrid,POINT (4499728.13308047 -407962.4875817323)
...,...,...,...,...,...
1840,Wamba,Pza Oriente,Forma parte esta escultura de la serie concebi...,Monumentos de la ciudad de Madrid,POINT (4499284.447377896 -413608.0839695063)
1841,Wifredo. El Velloso,Pza Oriente,Forma parte esta escultura de la serie concebi...,Monumentos de la ciudad de Madrid,POINT (4499398.509382567 -413488.3182058548)
1842,X aniversario Colegio Mayor Argentino Nuestra ...,C Martín Fierro,"La lápida, promovida por el Ministerio de Cult...",Monumentos de la ciudad de Madrid,POINT (4501320.015285811 -415743.259706655)
1843,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,"La lápida, promovida por la Embajada de la Rep...",Monumentos de la ciudad de Madrid,POINT (4501329.073807199 -415738.0970327701)


### Joining Data

Joining both tables with all posibilities for calculating the distance:

In [100]:
interest_points_bicimad = bicimad_stations.assign(key=0).merge(interest_points.assign(key=0), how='left', on = 'key')

In [101]:
interest_points_bicimad.drop('key', axis=1, inplace=True)

In [102]:
interest_points_bicimad

,Station location,BiciMAD station,dock_bikes,free_bases,coordinates_bicimad,Place of interest,Place address,Place Description,Type of place,coordinates_interest_points
0,Puerta del Sol nº 1,Puerta del Sol A,18,8,POINT (4499223.648366921 -412373.2851572837),A las víctimas del Holocausto,JARDIN TRES CULTURAS,"La obra, creada por el escultor judío de orige...",Monumentos de la ciudad de Madrid,POINT (4504467.690585745 -401410.9402809891)
1,Puerta del Sol nº 1,Puerta del Sol A,18,8,POINT (4499223.648366921 -412373.2851572837),A los Abuelos,C Alicún,"Pequeña escultura, situada en un parque delimi...",Monumentos de la ciudad de Madrid,POINT (4506192.045251724 -407898.4486445754)
2,Puerta del Sol nº 1,Puerta del Sol A,18,8,POINT (4499223.648366921 -412373.2851572837),A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumento dedicado al escritor Pablo Neruda (1...,Monumentos de la ciudad de Madrid,POINT (4495287.585088818 -406861.0266344676)
3,Puerta del Sol nº 1,Puerta del Sol A,18,8,POINT (4499223.648366921 -412373.2851572837),Abogados Laboristas,C Atocha 55,El 24 de enero de 1977 se produjo un atentado ...,Monumentos de la ciudad de Madrid,POINT (4498744.636240896 -412170.5648424985)
4,Puerta del Sol nº 1,Puerta del Sol A,18,8,POINT (4499223.648366921 -412373.2851572837),Abstracta I,,"La histórica quinta de la Fuente del Berro, de...",Monumentos de la ciudad de Madrid,POINT (4499728.13308047 -407962.4875817323)
...,...,...,...,...,...,...,...,...,...,...
473083,Avenida Complutense nº 23,Facultad Derecho,19,4,POINT (4502994.74083288 -415445.024322674),Wamba,Pza Oriente,Forma parte esta escultura de la serie concebi...,Monumentos de la ciudad de Madrid,POINT (4499284.447377896 -413608.0839695063)
473084,Avenida Complutense nº 23,Facultad Derecho,19,4,POINT (4502994.74083288 -415445.024322674),Wifredo. El Velloso,Pza Oriente,Forma parte esta escultura de la serie concebi...,Monumentos de la ciudad de Madrid,POINT (4499398.509382567 -413488.3182058548)
473085,Avenida Complutense nº 23,Facultad Derecho,19,4,POINT (4502994.74083288 -415445.024322674),X aniversario Colegio Mayor Argentino Nuestra ...,C Martín Fierro,"La lápida, promovida por el Ministerio de Cult...",Monumentos de la ciudad de Madrid,POINT (4501320.015285811 -415743.259706655)
473086,Avenida Complutense nº 23,Facultad Derecho,19,4,POINT (4502994.74083288 -415445.024322674),XXV aniversario Colegio Mayor Argentino,C Martín Fierro,"La lápida, promovida por la Embajada de la Rep...",Monumentos de la ciudad de Madrid,POINT (4501329.073807199 -415738.0970327701)


### Calculating distance

In [103]:
%%time
distance=interest_points_bicimad.apply(lambda x: distance_meters(x['coordinates_interest_points'],
                                                              x['coordinates_bicimad']), axis=1)

CPU times: user 6.09 s, sys: 36.8 ms, total: 6.13 s
Wall time: 6.13 s


In [104]:
interest_points_bicimad["distance"]=distance

Deleting not needed columns:

In [105]:
interest_points_bicimad.drop(['coordinates_bicimad','coordinates_interest_points'], axis=1, inplace=True)

Sorting values:

In [106]:
interest_points_bicimad=interest_points_bicimad.sort_values(["Place of interest","Station location"])

In [107]:
interest_points_bicimad

,Station location,BiciMAD station,dock_bikes,free_bases,Place of interest,Place address,Place Description,Type of place,distance
437250,Avenida Bruselas nº 56,Avenida Bruselas,4,18,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumento dedicado al escritor Pablo Neruda (1...,Monumentos de la ciudad de Madrid,6431.780052
471298,Avenida Complutense nº 23,Facultad Derecho,19,4,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumento dedicado al escritor Pablo Neruda (1...,Monumentos de la ciudad de Madrid,11536.258751
464130,Avenida Juan de Herrera frente a la calle Paul...,INEF,6,17,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumento dedicado al escritor Pablo Neruda (1...,Monumentos de la ciudad de Madrid,10735.369376
421122,Avenida Marqués de Corbera nº 12,Marqués de Corbera 12,8,15,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumento dedicado al escritor Pablo Neruda (1...,Monumentos de la ciudad de Madrid,4589.330269
422914,Avenida Marqués de Corbera nº 52,Marqués de Corbera 52,9,15,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumento dedicado al escritor Pablo Neruda (1...,Monumentos de la ciudad de Madrid,4939.620431
...,...,...,...,...,...,...,...,...,...
64163,Plaza del Cordón,Plaza del Cordón,13,11,Órgano,C Castelló 77,En 1971 el arquitecto José Luis Picardo ganó e...,Monumentos de la ciudad de Madrid,3759.110762
1443,Puerta del Sol nº 1,Puerta del Sol A,18,8,Órgano,C Castelló 77,En 1971 el arquitecto José Luis Picardo ganó e...,Monumentos de la ciudad de Madrid,2776.677372
3235,Puerta del Sol nº 1,Puerta del Sol B,21,7,Órgano,C Castelló 77,En 1971 el arquitecto José Luis Picardo ganó e...,Monumentos de la ciudad de Madrid,2749.128671
92835,Ronda de Atocha nº 2,Ronda de Atocha,14,9,Órgano,C Castelló 77,En 1971 el arquitecto José Luis Picardo ganó e...,Monumentos de la ciudad de Madrid,2962.047768


### Filtering by the minimun distant:

In [108]:
nearest_BiciMAD_station=interest_points_bicimad[interest_points_bicimad['distance'] ==
                        interest_points_bicimad.groupby(['Place of interest','Place address'])['distance'].transform('min')]

In [109]:
nearest_BiciMAD_station

,Station location,BiciMAD station,dock_bikes,free_bases,Place of interest,Place address,Place Description,Type of place,distance
410370,Calle Arroyo del Olivar nº 6,Concordia,22,1,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumento dedicado al escritor Pablo Neruda (1...,Monumentos de la ciudad de Madrid,2012.572727
415744,Calle Carlota O'Neill nº 32,Carlota O’Neill,7,17,A las víctimas del Holocausto,JARDIN TRES CULTURAS,"La obra, creada por el escultor judío de orige...",Monumentos de la ciudad de Madrid,5559.887276
449793,Paseo de la Castellana nº 298,Manuel Caldeiro,6,18,A los Abuelos,C Alicún,"Pequeña escultura, situada en un parque delimi...",Monumentos de la ciudad de Madrid,2633.107767
164883,Calle Goya nº 1,Plaza de Colón,1,20,AICO,Pza Colón,Como recordatorio de las fructíferas relacione...,Monumentos de la ciudad de Madrid,110.126957
73475,Calle Atocha nº 54,Antón Martín,12,4,Abogados Laboristas,C Atocha 55,El 24 de enero de 1977 se produjo un atentado ...,Monumentos de la ciudad de Madrid,127.684089
...,...,...,...,...,...,...,...,...,...
62769,Plaza del Cordón,Plaza del Cordón,13,11,Álvaro de Bazán,Pza Villa,"Álvaro de Bazán, primer marqués de Santa Cruz ...",Monumentos de la ciudad de Madrid,102.328354
453441,Calle Santa Hortensia nº 31,Corazón de María,4,19,Ángeles de la Paz,Avda Bruselas,"Este monumento, proyectado en chapa de hierro ...",Monumentos de la ciudad de Madrid,326.569317
415829,Calle Carlota O'Neill nº 32,Carlota O’Neill,7,17,Árbol de la Vida,JARDIN TRES CULTURAS,El Jardín de las Tres Culturas es un ámbito es...,Monumentos de la ciudad de Madrid,5559.887276
65596,Plaza de Ramales,Plaza Ramales,9,15,Íñigo Arista,Pza Oriente,Forma parte esta escultura de la serie concebi...,Monumentos de la ciudad de Madrid,108.278072


Example for testing:

In [110]:
nearest_BiciMAD_station[(nearest_BiciMAD_station["Place of interest"]=="A las víctimas del Holocausto") | (nearest_BiciMAD_station["Place of interest"]=="A los Abuelos")]

,Station location,BiciMAD station,dock_bikes,free_bases,Place of interest,Place address,Place Description,Type of place,distance
415744,Calle Carlota O'Neill nº 32,Carlota O’Neill,7,17,A las víctimas del Holocausto,JARDIN TRES CULTURAS,"La obra, creada por el escultor judío de orige...",Monumentos de la ciudad de Madrid,5559.887276
449793,Paseo de la Castellana nº 298,Manuel Caldeiro,6,18,A los Abuelos,C Alicún,"Pequeña escultura, situada en un parque delimi...",Monumentos de la ciudad de Madrid,2633.107767


Sorting the data:

In [111]:
nearest_BiciMAD_station=nearest_BiciMAD_station.sort_values(["Place of interest","Station location"])
nearest_BiciMAD_station=nearest_BiciMAD_station.reset_index()

In [112]:
nearest_BiciMAD_station

,index,Station location,BiciMAD station,dock_bikes,free_bases,Place of interest,Place address,Place Description,Type of place,distance
0,410370,Calle Arroyo del Olivar nº 6,Concordia,22,1,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumento dedicado al escritor Pablo Neruda (1...,Monumentos de la ciudad de Madrid,2012.572727
1,415744,Calle Carlota O'Neill nº 32,Carlota O’Neill,7,17,A las víctimas del Holocausto,JARDIN TRES CULTURAS,"La obra, creada por el escultor judío de orige...",Monumentos de la ciudad de Madrid,5559.887276
2,449793,Paseo de la Castellana nº 298,Manuel Caldeiro,6,18,A los Abuelos,C Alicún,"Pequeña escultura, situada en un parque delimi...",Monumentos de la ciudad de Madrid,2633.107767
3,164883,Calle Goya nº 1,Plaza de Colón,1,20,AICO,Pza Colón,Como recordatorio de las fructíferas relacione...,Monumentos de la ciudad de Madrid,110.126957
4,73475,Calle Atocha nº 54,Antón Martín,12,4,Abogados Laboristas,C Atocha 55,El 24 de enero de 1977 se produjo un atentado ...,Monumentos de la ciudad de Madrid,127.684089
...,...,...,...,...,...,...,...,...,...,...
1781,62769,Plaza del Cordón,Plaza del Cordón,13,11,Álvaro de Bazán,Pza Villa,"Álvaro de Bazán, primer marqués de Santa Cruz ...",Monumentos de la ciudad de Madrid,102.328354
1782,453441,Calle Santa Hortensia nº 31,Corazón de María,4,19,Ángeles de la Paz,Avda Bruselas,"Este monumento, proyectado en chapa de hierro ...",Monumentos de la ciudad de Madrid,326.569317
1783,415829,Calle Carlota O'Neill nº 32,Carlota O’Neill,7,17,Árbol de la Vida,JARDIN TRES CULTURAS,El Jardín de las Tres Culturas es un ámbito es...,Monumentos de la ciudad de Madrid,5559.887276
1784,65596,Plaza de Ramales,Plaza Ramales,9,15,Íñigo Arista,Pza Oriente,Forma parte esta escultura de la serie concebi...,Monumentos de la ciudad de Madrid,108.278072


In [113]:
column_names = ["Place of interest", "Type of place", "Place address",
                "BiciMAD station","Station location","distance","dock_bikes","free_bases","Place Description"]
nearest_BiciMAD_station=nearest_BiciMAD_station.reindex(columns=column_names)

In [114]:
nearest_BiciMAD_station = nearest_BiciMAD_station.rename(columns={'dock_bikes': 'bikes availability','distance': 'distance(m)','free_bases': 'bases availability'})

In [115]:
nearest_BiciMAD_station

,Place of interest,Type of place,Place address,BiciMAD station,Station location,distance(m),bikes availability,bases availability,Place Description
0,A Neruda-El Ser alado,Monumentos de la ciudad de Madrid,Gta José Luis Ozores 116,Concordia,Calle Arroyo del Olivar nº 6,2012.572727,22,1,Monumento dedicado al escritor Pablo Neruda (1...
1,A las víctimas del Holocausto,Monumentos de la ciudad de Madrid,JARDIN TRES CULTURAS,Carlota O’Neill,Calle Carlota O'Neill nº 32,5559.887276,7,17,"La obra, creada por el escultor judío de orige..."
2,A los Abuelos,Monumentos de la ciudad de Madrid,C Alicún,Manuel Caldeiro,Paseo de la Castellana nº 298,2633.107767,6,18,"Pequeña escultura, situada en un parque delimi..."
3,AICO,Monumentos de la ciudad de Madrid,Pza Colón,Plaza de Colón,Calle Goya nº 1,110.126957,1,20,Como recordatorio de las fructíferas relacione...
4,Abogados Laboristas,Monumentos de la ciudad de Madrid,C Atocha 55,Antón Martín,Calle Atocha nº 54,127.684089,12,4,El 24 de enero de 1977 se produjo un atentado ...
...,...,...,...,...,...,...,...,...,...
1781,Álvaro de Bazán,Monumentos de la ciudad de Madrid,Pza Villa,Plaza del Cordón,Plaza del Cordón,102.328354,13,11,"Álvaro de Bazán, primer marqués de Santa Cruz ..."
1782,Ángeles de la Paz,Monumentos de la ciudad de Madrid,Avda Bruselas,Corazón de María,Calle Santa Hortensia nº 31,326.569317,4,19,"Este monumento, proyectado en chapa de hierro ..."
1783,Árbol de la Vida,Monumentos de la ciudad de Madrid,JARDIN TRES CULTURAS,Carlota O’Neill,Calle Carlota O'Neill nº 32,5559.887276,7,17,El Jardín de las Tres Culturas es un ámbito es...
1784,Íñigo Arista,Monumentos de la ciudad de Madrid,Pza Oriente,Plaza Ramales,Plaza de Ramales,108.278072,9,15,Forma parte esta escultura de la serie concebi...


In [116]:
nearest_BiciMAD_station['distance(m)']=nearest_BiciMAD_station['distance(m)'].round(2)

In [117]:
nearest_BiciMAD_station

,Place of interest,Type of place,Place address,BiciMAD station,Station location,distance(m),bikes availability,bases availability,Place Description
0,A Neruda-El Ser alado,Monumentos de la ciudad de Madrid,Gta José Luis Ozores 116,Concordia,Calle Arroyo del Olivar nº 6,2012.57,22,1,Monumento dedicado al escritor Pablo Neruda (1...
1,A las víctimas del Holocausto,Monumentos de la ciudad de Madrid,JARDIN TRES CULTURAS,Carlota O’Neill,Calle Carlota O'Neill nº 32,5559.89,7,17,"La obra, creada por el escultor judío de orige..."
2,A los Abuelos,Monumentos de la ciudad de Madrid,C Alicún,Manuel Caldeiro,Paseo de la Castellana nº 298,2633.11,6,18,"Pequeña escultura, situada en un parque delimi..."
3,AICO,Monumentos de la ciudad de Madrid,Pza Colón,Plaza de Colón,Calle Goya nº 1,110.13,1,20,Como recordatorio de las fructíferas relacione...
4,Abogados Laboristas,Monumentos de la ciudad de Madrid,C Atocha 55,Antón Martín,Calle Atocha nº 54,127.68,12,4,El 24 de enero de 1977 se produjo un atentado ...
...,...,...,...,...,...,...,...,...,...
1781,Álvaro de Bazán,Monumentos de la ciudad de Madrid,Pza Villa,Plaza del Cordón,Plaza del Cordón,102.33,13,11,"Álvaro de Bazán, primer marqués de Santa Cruz ..."
1782,Ángeles de la Paz,Monumentos de la ciudad de Madrid,Avda Bruselas,Corazón de María,Calle Santa Hortensia nº 31,326.57,4,19,"Este monumento, proyectado en chapa de hierro ..."
1783,Árbol de la Vida,Monumentos de la ciudad de Madrid,JARDIN TRES CULTURAS,Carlota O’Neill,Calle Carlota O'Neill nº 32,5559.89,7,17,El Jardín de las Tres Culturas es un ámbito es...
1784,Íñigo Arista,Monumentos de la ciudad de Madrid,Pza Oriente,Plaza Ramales,Plaza de Ramales,108.28,9,15,Forma parte esta escultura de la serie concebi...


### Saving a csv with the results:

In [178]:
nearest_BiciMAD_station.to_csv('../data/output/nearest_BiciMAD_station.csv')

### Function for selecting one interesting place

In [179]:
def nearest_station_specific_place(place):
    nearest_BiciMAD_station_filtered=nearest_BiciMAD_station[nearest_BiciMAD_station["Place of interest"]==place]
    nearest_BiciMAD_station_filtered.to_csv('../data/output/nearest_BiciMAD_station_filtered.csv')
    return nearest_BiciMAD_station_filtered

Example:

In [180]:
nearest_station_specific_place(place='A los Abuelos')

,Place of interest,Type of place,Place address,BiciMAD station,Station location,distance(m),bikes availability,bases availability,Place Description,similarity
2,A los Abuelos,Monumentos de la ciudad de Madrid,C Alicún,Manuel Caldeiro,Paseo de la Castellana nº 298,2633.11,6,18,"Pequeña escultura, situada en un parque delimi...",40


### Using fuzzywuzzy

In [181]:
#Using an example for testing functionality
input_var='victimas del holocausto'

In [182]:
def compare_strings(input_var,x):
    return fuzz.partial_ratio(input_var,x)

In [183]:
similarity=nearest_BiciMAD_station.apply(lambda x: compare_strings(x['Place of interest'],input_var), axis=1)

In [184]:
similarity

0       32
1       91
2       40
3        0
4       32
        ..
1781    40
1782    41
1783    39
1784    25
1785    33
Length: 1786, dtype: int64

In [185]:
nearest_BiciMAD_station["similarity"]=similarity

In [186]:
nearest_BiciMAD_station[nearest_BiciMAD_station["similarity"]>80]

,Place of interest,Type of place,Place address,BiciMAD station,Station location,distance(m),bikes availability,bases availability,Place Description,similarity
1,A las víctimas del Holocausto,Monumentos de la ciudad de Madrid,JARDIN TRES CULTURAS,Carlota O’Neill,Calle Carlota O'Neill nº 32,5559.89,7,17,"La obra, creada por el escultor judío de orige...",91
